In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json

In [2]:
now = dt.datetime.now()
date_today = str(now.day)+'-'+str(now.month)+'-'+str(now.year)
date_start = '02-01-2019'

# In case of scraping the web

In [2]:
#base_html = 'C://Users//bruno//Documents//DS//DataSets Grandes//Historicos Economicos//MEP Historico.html'
base_html = 'H://My Drive//Formacion//Data Science//Financial//FinancialPF//Web Sites//MEP Histórico.html'

Fuente: https://www.ambito.com/contenidos/dolar-mep-historico.html

In [3]:
mep_hist = pd.read_html(base_html) # this parses all the tables in webpages to a list
df_MEP = mep_hist[0]
df_MEP.head() # Values are int without decimals, which is incorrect due to the real value's nature

,Fecha,Referencia
0,05-11-2021,18126
1,04-11-2021,18056
2,03-11-2021,18007
3,02-11-2021,17932
4,01-11-2021,17922


In [4]:
def decimals(df, column='Value'):
    '''Converts integer value into float considering last 2 numbers as decimals. df parameter to specify df, column parameter to specify target column of df'''
    new_value_list = []
    for value in df[column]:
        ss1, ss2 = str(value)[:-2], str(value)[-2:] # Slicing the value as string to get separated the number
        new = ss1+'.'+ss2 # Adding the . into the value
        new_value_list.append(float(new)) # Adding the new value to a list}
    df[column] = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values

In [5]:
df_MEP.columns = ["Date", 'Value']

In [6]:
decimals(df_MEP, 'Value')

In [7]:
df_MEP.head(5)

,Date,Value
0,05-11-2021,181.26
1,04-11-2021,180.56
2,03-11-2021,180.07
3,02-11-2021,179.32
4,01-11-2021,179.22


In [8]:
df_MEP.tail(5) # Checking last values with changes made

,Date,Value
591,24-03-2020,91.4
592,23-03-2020,91.4
593,22-03-2020,91.4
594,21-03-2020,91.4
595,20-03-2020,91.4


In [9]:
df_MEP.to_csv('H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\MEP.csv', index=False)

# In case of using the web's API

In [14]:
# API Ámbito
request_url_oficial = 'https://mercados.ambito.com//dolar/mep/historico-general/' #20-10-2021/20-11-2021'
request_url = 'https://mercados.ambito.com//dolarrava/mep/historico-general/' #20-10-2021/20-11-2021'

request_url = request_url+date_start+'/'+date_today
request_url_oficial = request_url_oficial+date_start+'/'+date_today

In [15]:
response = requests.get(request_url)
response_oficial = requests.get(request_url_oficial)

In [16]:
response = response.text[1:]
response = response[:-1]
response = response.replace('],[', '];[')
response_oficial = response_oficial.text[1:]
response_oficial = response_oficial[:-1]
response_oficial = response_oficial.replace('],[', '];[')

In [17]:
(response).split(';')[10].replace('","', '";"') # Identifying tuples
(response_oficial).split(';')[10].replace('","', '";"') # Identifying tuples

'["08-11-2021";"99,58";"105,58"]'

In [18]:
(response).split(';')[10].replace('","', '";"').replace('[','').replace(']','').replace('"','').split(';') # Splitting values
(response_oficial).split(';')[10].replace('","', '";"').replace('[','').replace(']','').replace('"','').split(';') # Splitting values

['08-11-2021', '99,58', '105,58']

In [19]:
prices = []
for i in range (0,1000):
    try:
        prices.append(((response).split(';')[i].replace('","', '";"').replace('[','').replace(']','').replace('"','').replace(',','.').split(';')))
    except:
        pass

prices_oficial = []
for i in range (0,1000):
    try:
        prices_oficial.append(((response_oficial).split(';')[i].replace('","', '";"').replace('[','').replace(']','').replace('"','').replace(',','.').split(';')))
    except:
        pass

In [24]:
MEP = pd.DataFrame(prices[1:], columns=prices[0])
MEP.Referencia = MEP.Referencia.astype('float')
Oficial = pd.DataFrame(prices_oficial[1:], columns=prices_oficial[0])
Oficial.Venta = Oficial.Venta.astype('float')

In [28]:
MEP.columns = ['Fecha', 'Venta']

In [25]:
MEP.drop_duplicates(subset=['Fecha'], keep='first', inplace=True)
Oficial.drop_duplicates(subset=['Fecha'], keep='first', inplace=True)

In [30]:
path = 'H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\\'
MEP.to_csv(path+'MEP_API.csv', index=False)
Oficial.to_csv(path+'Oficial_API.csv', index=False)